Удалила "2 Зрелый почерк-(обычный почерк)-2.jpg" (там печатный текст)

# Config

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/MyDrive/Colab\ Notebooks/mest

/content/drive/MyDrive/Colab Notebooks/mest


In [3]:
!pip install hydra-core --upgrade

In [4]:
from src.commands import *
import hydra

hydra.core.global_hydra.GlobalHydra.instance().clear()
CONF = hydra_config("config2.yaml")
show_CONF(CONF)

{
        "seed": 42,
        "device": "cuda",
        "save_path": "checkpoints",
        "dataset": {
                "data_pos": "bin/1",
                "data_neg": "bin/0",
                "threshold": 0.999,
                "pos_aug": 22,
                "neg_aug": 2
        },
        "loader": {
                "batch_size": 8
        },
        "classifier": {
                "n_epochs": 30,
                "learning_rate": 0.0001,
                "train": true,
                "path_model": "resnet",
                "path_log": "resnet.pickle"
        }
}


In [5]:
show_CONF(CONF)

{
        "seed": 42,
        "device": "cuda",
        "save_path": "checkpoints",
        "dataset": {
                "data_pos": "bin/1",
                "data_neg": "bin/0",
                "threshold": 0.999,
                "pos_aug": 22,
                "neg_aug": 2
        },
        "loader": {
                "batch_size": 8
        },
        "classifier": {
                "n_epochs": 30,
                "learning_rate": 0.0001,
                "train": true,
                "path_model": "resnet",
                "path_log": "resnet.pickle"
        }
}


# Библиотеки

In [6]:
from hydra import compose, initialize
from omegaconf import OmegaConf
import json, os
import matplotlib.pyplot as plt

from torchvision import datasets, models, transforms
from time import time
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import torch
import os, json, cv2, re
import numpy as np

from src.dataset import My_Dataset, TripletDataset, Emb_Dataset
from src.functions import number_of_parameters, seed_everything, create_model
from src.dataset_fun import split_dataset
from src import siamese, classifier
from src.classifier import get_predictions
from src.train import train, load_logs, load_model
from src.functions import confusion_matrix, distribution
from src.show_fun import show_result, show_images

# train

In [7]:
pos_dataset, neg_dataset = pos_neg_dataset(CONF)

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_classes = 2
num_non_freeze = 2361346
seed_everything(CONF.seed)
model = create_model(model, num_non_freeze, num_classes).to(CONF.device)
print(f"num parameters = {number_of_parameters(model)}")

dataset = torch.utils.data.ConcatDataset([pos_dataset, neg_dataset])
seed_everything(CONF.seed)
train_dataset, test_dataset = split_dataset(dataset)

common_train(CONF, model, train_dataset, test_dataset, kind="cl")
load_best_model(CONF, model, train_dataset, test_dataset, kind="cl")

Positive dataset: 484    (22)
Negative dataset: 448    (224)


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

# eval

In [ ]:
X, y_true, y_pred, y_prob = get_predictions(model, pos_dataset,
                                            CONF.loader.batch_size, CONF.device)
print(accuracy_score(y_pred, y_true))
# 0.987603305785124
# array([[  0,   0],
#        [  6, 478]])
print(confusion_matrix(y_true, y_pred))
distribution(y_prob)

In [ ]:
X, y_true, y_pred, y_prob = get_predictions(model, neg_dataset,
                                            CONF.loader.batch_size, CONF.device)
print(accuracy_score(y_pred, y_true))
# 0.96875
# array([[435,  13],
#        [  0,   0]])
print(confusion_matrix(y_true, y_pred))
distribution(y_prob)

In [ ]:
X, y_true, y_pred, y_prob = get_predictions(model, dataset,
                                            CONF.loader.batch_size, CONF.device)
print(accuracy_score(y_pred, y_true))
print(confusion_matrix(y_true, y_pred))
# 0.9785407725321889
# array([[435,  13],
#        [  6, 478]])

# show

In [ ]:
X, y_true, y_pred, y_prob, indices = show_result(model, pos_dataset, 0.6,
                                   CONF.loader.batch_size, CONF.device,
                                   greater = False, col = 8,
                                   sort = True)

In [ ]:
X, y_true, y_pred, y_prob, indices = show_result(model, neg_dataset, 0.4,
                                   CONF.loader.batch_size, CONF.device,
                                   greater = True, col = 24,
                                   sort = True)

# on the entire dataset

In [ ]:
X, y_true, y_pred, y_prob, indices = show_result(model, neg_dataset, 0,
                                   CONF.loader.batch_size, CONF.device,
                                   greater = True, col = 448,
                                   sort = True)

In [ ]:
X, y_true, y_pred, y_prob, indices = show_result(model, pos_dataset, 0,
                                   CONF.loader.batch_size, CONF.device,
                                   greater = True, col = 484,
                                   sort = True)